# Zero to Hero: Amazon Forecast

This tutorial is to provide a quick technical introduction on Machine Learning using Amazon Forecast.
Base Forecast instructions were taken from https://github.com/aws-samples/amazon-forecast-samples/tree/main/notebooks/basic/Getting_Started

<img title="AWS Forecast" alt="Seeing the future" src="./images/AWSForecast.jpeg">

To bring more flavour and appreciation on this topic, we will be using Forecast to predict the Sales on a particular shop/retail store.

## Business Background:
- Client owns a shop/retail store in the US (as we will be leveraging US holidays) and currently projecting sales using primitive rolling 52 week approach.
  - Link on this approach https://www.lawinsider.com/dictionary/52-week-rolling-forecast
- Client is trying to project a more realistic forecast

## Solution:
- We'll use Machine Learning algorithm leveraging Amazon Forecast to get a more accurate sales number

## Knowledge Pre-requisite:
- Good understanding on Jupyter Notebook
- Python Programming background

We will start by **importing the historical data from 2002 to 3rd September 2023**. Next, we will **train a Predictor** using this data. Finally, we will **generate a forecast for September and October 2023** and **compare it with the actual data - which is the ground Truth/Reality that we've captured**.

## High-Level Approach
* [Loading Required Notebook Modules](#loadingmodules)
* Step 1: [Import your data](#import)
* Step 2: [Train a predictor](#predictor)
* Step 3: [Generate forecasts](#forecast)
* [Clean-up](#cleanup)

## Loading Required Notebook Modules <a class="anchor" id="loadingmodules"></a>
To ensure we can run the required commands on the notebook environment, the AWS SDK client for Amazon Forecast and IAM Role used by Amazon Forecast to access your data.

#### Setup Notebook Environment

In [ ]:
%%capture --no-stderr setup

!pip install pandas s3fs matplotlib ipywidgets
!pip install boto3 --upgrade

%reload_ext autoreload

#### Setup Imports
The util function is taken from https://github.com/aws-samples/amazon-forecast-samples/tree/main/notebooks/common

In [ ]:
import sys
import os

sys.path.insert( 0, os.path.abspath("/") )

import json
import util
import boto3
import s3fs
import pandas as pd

#### Create an instance of AWS SDK client for Amazon Forecast

In [ ]:
region = 'us-east-1'
session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast')
forecastquery = session.client(service_name='forecastquery')

# Checking to make sure we can communicate with Amazon Forecast
assert forecast.list_predictors()

#### Setup IAM Role used by Amazon Forecast to access your data

In [ ]:
role_name = "ForecastNotebookRole-Basic"
print(f"Creating Role {role_name}...")
role_arn = util.get_or_create_iam_role( role_name = role_name )

# echo user inputs without account
print(f"Success! Created role = {role_arn.split('/')[1]}")

## Step 1: Import your data. <a class="anchor" id="import"></a>

In this step, we will create a **Dataset** and **Import** from our data to S3 to Amazon Forecast. To train a Predictor we will need a **DatasetGroup** that groups the input **Datasets**. So, we will end this step by creating a **DatasetGroup** with the imported **Dataset**.

#### Peek at the data and upload it to S3.

The Sales dataset has the following 3 columns:
1. **timestamp:** Timetamp at which the sales date.
2. **item_id:** Shop/Retail Store ID.
3. **target_value:** Total sales for the day.

In [ ]:
key="data/training-data/200204-202308-sales-data.csv"

sales_df = pd.read_csv(key, dtype = object, names=['timestamp','item_id','target_value'])

display(sales_df.head(3))

bucket_name = input("\nEnter S3 bucket name for uploading the data and hit ENTER key:")
print(f"\nAttempting to upload the data to the S3 bucket '{bucket_name}' at key '{key}' ...")

s3 = boto3.Session().resource('s3')
bucket = s3.Bucket(bucket_name)
if not bucket.creation_date:
    if region != "us-east-1":
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
    else:
        s3.create_bucket(Bucket=bucket_name)

s3.Bucket(bucket_name).Object(key).upload_file(key)
ts_s3_path = f"s3://{bucket_name}/{key}"

print(f"\nDone, the dataset is uploaded to S3 at {ts_s3_path}.")

#### Creating the Dataset

In [ ]:
DATASET_FREQUENCY = "D" # D for Daily.
TS_DATASET_NAME = "SALES_TS" # Sales Time Series
TS_SCHEMA = {
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      },
      {
         "AttributeName":"target_value",
         "AttributeType":"float"
      }
   ]
}

create_dataset_response = forecast.create_dataset(Domain="CUSTOM",
                                                  DatasetType='TARGET_TIME_SERIES',
                                                  DatasetName=TS_DATASET_NAME,
                                                  DataFrequency=DATASET_FREQUENCY,
                                                  Schema=TS_SCHEMA)

ts_dataset_arn = create_dataset_response['DatasetArn']
describe_dataset_response = forecast.describe_dataset(DatasetArn=ts_dataset_arn)

print(f"The Dataset with ARN {ts_dataset_arn} is now {describe_dataset_response['Status']}.")

# Importing the Dataset
This will take 5-10 minutes to import the data.

In [ ]:
TIMESTAMP_FORMAT = "yyyy-MM-dd"
TS_IMPORT_JOB_NAME = "SALES_TTS_IMPORT"
TIMEZONE = "EST"

ts_dataset_import_job_response = \
    forecast.create_dataset_import_job(DatasetImportJobName=TS_IMPORT_JOB_NAME,
                                       DatasetArn=ts_dataset_arn,
                                       DataSource= {
                                         "S3Config" : {
                                             "Path": ts_s3_path,
                                             "RoleArn": role_arn
                                         } 
                                       },
                                       TimestampFormat=TIMESTAMP_FORMAT,
                                       TimeZone = TIMEZONE)

ts_dataset_import_job_arn = ts_dataset_import_job_response['DatasetImportJobArn']
describe_dataset_import_job_response = forecast.describe_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn)

print(f"Waiting for Dataset Import Job with ARN {ts_dataset_import_job_arn} to become ACTIVE. This process could take 5-10 minutes.\n\nCurrent Status:")

status = util.wait(lambda: forecast.describe_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn))

describe_dataset_import_job_response = forecast.describe_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn)
print(f"\n\nThe Dataset Import Job with ARN {ts_dataset_import_job_arn} is now {describe_dataset_import_job_response['Status']}.")

#### Creating a DatasetGroup

In [ ]:
DATASET_GROUP_NAME = "SALES_DEMO"
DATASET_ARNS = [ts_dataset_arn]

create_dataset_group_response = \
    forecast.create_dataset_group(Domain="CUSTOM",
                                  DatasetGroupName=DATASET_GROUP_NAME,
                                  DatasetArns=DATASET_ARNS)

dataset_group_arn = create_dataset_group_response['DatasetGroupArn']
describe_dataset_group_response = forecast.describe_dataset_group(DatasetGroupArn=dataset_group_arn)

print(f"The DatasetGroup with ARN {dataset_group_arn} is now {describe_dataset_group_response['Status']}.")

## You can check the Dataset using the Forecast UI in AWS Console.
<img title="AWS Forecast Target Time Series Dataset View" alt="AWS Forecast Target Time Series Dataset View" src="/images/target-time-series-dataset.png">

## Step 2: Train a predictor <a class="anchor" id="predictor"></a>

In this step, we will create a **Predictor** using the **DatasetGroup** that was created above. After creating the predictor, we will review the accuracy obtained through the backtesting process to get a quantitative understanding of the performance of the predictor.

#### Train a predictor
This would take 3-4 hours.

<img title="AWS Predictors Training" alt="AWS Predictors Training" src="/images/aws-predictors-training.png">

In [ ]:
PREDICTOR_NAME = "SALES_PREDICTOR"
FORECAST_HORIZON = 14 # 1 quarter, don't like 13 so made it 14 weeks
FORECAST_FREQUENCY = "W"
HOLIDAY_DATASET = [{
        'Name': 'holiday',
        'Configuration': {
        'CountryCode': ['US']
    }
}]

create_auto_predictor_response = \
    forecast.create_auto_predictor(PredictorName = PREDICTOR_NAME,
                                   ForecastHorizon = FORECAST_HORIZON,
                                   ForecastFrequency = FORECAST_FREQUENCY,
                                   DataConfig = {
                                       'DatasetGroupArn': dataset_group_arn, 
                                       'AdditionalDatasets': HOLIDAY_DATASET
                                    },
                                   ExplainPredictor = True)

predictor_arn = create_auto_predictor_response['PredictorArn']
print(f"Waiting for Predictor with ARN {predictor_arn} to become ACTIVE. Depending on data size and predictor setting，it can take several hours to be ACTIVE.\n\nCurrent Status:")

status = util.wait(lambda: forecast.describe_auto_predictor(PredictorArn=predictor_arn))

describe_auto_predictor_response = forecast.describe_auto_predictor(PredictorArn=predictor_arn)
print(f"\n\nThe Predictor with ARN {predictor_arn} is now {describe_auto_predictor_response['Status']}.")

#### Review accuracy metrics

* **Weighted Quantile Loss (wQL)** metric measures the accuracy of a model at a specified quantile. It is particularly useful when there are different costs for underpredicting and overpredicting.

* **Root Mean Square Error (RMSE)** uses the squared value of the residuals, which amplifies the impact of outliers. In use cases where only a few large mispredictions can be very costly, the RMSE is the more relevant metric.

* **Weighted Absolute Percentage Error (WAPE)** is more robust to outliers than Root Mean Square Error (RMSE) because it uses the absolute error instead of the squared error.

* **Mean Absolute Percentage Error (MAPE)** is useful for cases where values differ significantly between time points and outliers have a significant impact.

* **Mean Absolute Scaled Error (MASE)** is ideal for datasets that are cyclical in nature or have seasonal properties.

In [ ]:
get_accuracy_metrics_response = forecast.get_accuracy_metrics(PredictorArn=predictor_arn)
wql = get_accuracy_metrics_response['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['WeightedQuantileLosses']
accuracy_scores = get_accuracy_metrics_response['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['ErrorMetrics'][0]

print(f"Weighted Quantile Loss (wQL): {json.dumps(wql, indent=2)}\n\n")

print(f"Root Mean Square Error (RMSE): {accuracy_scores['RMSE']}\n\n")

print(f"Weighted Absolute Percentage Error (WAPE): {accuracy_scores['WAPE']}\n\n")

print(f"Mean Absolute Percentage Error (MAPE): {accuracy_scores['MAPE']}\n\n")

print(f"Mean Absolute Scaled Error (MASE): {accuracy_scores['MASE']}\n")

## Step 3: Generate forecasts <a class="anchor" id="forecast"></a>
We will generate the forecasts using the above predictor. Later in this step we will also compare the forecast with the ground truth for September 2023 to demonstrate actual performance of Amazon Forecast on this dataset.

#### Generate forecasts

In [ ]:
FORECAST_NAME = "SALES_FORECAST"

create_forecast_response = \
    forecast.create_forecast(ForecastName=FORECAST_NAME,
                             PredictorArn=predictor_arn)

forecast_arn = create_forecast_response['ForecastArn']
print(f"Waiting for Forecast with ARN {forecast_arn} to become ACTIVE. Depending on data size and predictor settings，it can take several hours to be ACTIVE.\n\nCurrent Status:")

status = util.wait(lambda: forecast.describe_forecast(ForecastArn=forecast_arn))

describe_forecast_response = forecast.describe_forecast(ForecastArn=forecast_arn)
print(f"\n\nThe Forecast with ARN {forecast_arn} is now {describe_forecast_response['Status']}.")

#### Load ground truth for Sales Forecast in September 2023.

In [ ]:
ITEM_ID = "Shop1"

sales_sep2023_df = pd.read_csv("data/ground-truth/202309-sales-data.csv", dtype = object, names=['timestamp','item_id','target_value'])
sales_sep2023_df.target_value = sales_sep2023_df.target_value.astype(float)

actuals = sales_sep2023_df[(sales_sep2023_df['item_id'] == ITEM_ID)]

#### Query forecasts for Sales 2023.

In [ ]:
forecast_response = forecastquery.query_forecast(
    ForecastArn=forecast_arn,
    Filters={"item_id": ITEM_ID}
)

forecasts_p10_df = pd.DataFrame.from_dict(forecast_response['Forecast']['Predictions']['p10'])
forecasts_p50_df = pd.DataFrame.from_dict(forecast_response['Forecast']['Predictions']['p50'])
forecasts_p90_df = pd.DataFrame.from_dict(forecast_response['Forecast']['Predictions']['p90'])

#### Compare the forecasts with ground truth

In [ ]:
results_df = pd.DataFrame(columns=['timestamp', 'value', 'source'])

for index, row in actuals.iterrows():
    clean_timestamp = dateutil.parser.parse(row['timestamp'])
    results_df = results_df.append({'timestamp' : clean_timestamp , 'value' : row['target_value'], 'source': 'actual'} , ignore_index=True)
for index, row in forecasts_p10_df.iterrows():
    clean_timestamp = dateutil.parser.parse(row['Timestamp'])
    results_df = results_df.append({'timestamp' : clean_timestamp , 'value' : row['Value'], 'source': 'p10'} , ignore_index=True)
for index, row in forecasts_p50_df.iterrows():
    clean_timestamp = dateutil.parser.parse(row['Timestamp'])
    results_df = results_df.append({'timestamp' : clean_timestamp , 'value' : row['Value'], 'source': 'p50'} , ignore_index=True)
for index, row in forecasts_p90_df.iterrows():
    clean_timestamp = dateutil.parser.parse(row['Timestamp'])
    results_df = results_df.append({'timestamp' : clean_timestamp , 'value' : row['Value'], 'source': 'p90'} , ignore_index=True)

pivot_df = results_df.pivot(columns='source', values='value', index="timestamp")

pivot_df.plot(figsize=(15, 7))

## Clean-up <a class="anchor" id="cleanup"></a>
Uncomment the code section to delete all resources that were created in this notebook.

In [ ]:
# forecast.delete_resource_tree(ResourceArn = dataset_group_arn)
# forecast.delete_resource_tree(ResourceArn = ts_dataset_arn)